In [1]:
import pandas as pd
from IPython.display import display, HTML


import base
import fund_scanner.common_tools.database as db
import fund_scanner.business.grab_data as grab_data
engine = db.get_sqlalchemy_engine()

In [10]:
url = 'http://fund.eastmoney.com/manager/%s.html'

with db.get_connection() as cursor:
    sql = 'select * from managers where managers_code=%s order by funds_history_update_time limit 2 '
    cursor.execute(sql, '30037591')
    managers = cursor.fetchall()
    print(managers[0])
    
    for manager in managers:
        
        

        print(manager['managers_code'])

        df = pd.read_html(url%manager['managers_code'])[1]

        #len(df)

        if df.iloc[0,-1][-1] != '%' and df.iloc[0,-1][-1] != '-':
            print('抓取到的格式错误！')
            display(df)

        df['start_date'] = df.loc[:,'任职时间'].apply(lambda x: x.split(' ~ ')[0])
        df['end_date'] = df.loc[:,'任职时间'].apply(lambda x: x.split(' ~ ')[1].replace('至今', '9999-9-9'))
        df['funds_code'] = df.loc[:,'基金代码'].apply(lambda x: str(x).zfill(6))

        codes = df['基金代码'].tolist()
        codes = ','.join([str(w) for w in codes])

        df_funds = pd.read_sql('select * from funds where funds_code in (%s)'%(codes), engine)

        df = pd.merge(df, df_funds, on='funds_code', how='left')

        funds_need_update = df[df['funds_type'].isnull()]['funds_code'].tolist()
        if len(funds_need_update)>1:
            grab_data.update_funds(funds_need_update)

        df_funds = pd.read_sql('select * from funds where funds_code in (%s)'%(codes), engine)

        df = pd.merge(df, df_funds, on='funds_code', how='inner')

        display(df)
        
        for row in df.iterrows():
            start_date = row[1]['start_date']
            end_date = row[1]['end_date']
            managers_id = manager['managers_id']
            funds_id = int(row[1]['funds_id_x'])
            sql = 'INSERT INTO `managers_2_funds_history` (managers_id, funds_id, history_start_date, history_end_date) VALUES (%s, %s, %s, %s) '\
            'ON DUPLICATE KEY UPDATE history_end_date=%s;'
            print(sql % (managers_id, funds_id, start_date, end_date, end_date))
            cursor.execute(sql, (managers_id, funds_id, start_date, end_date, end_date))
            sql = 'UPDATE `managers` set funds_history_update_time=Now() where managers_id=%s;'
            cursor.execute(sql, (managers_id))
            




{'managers_name': '应帅', 'managers_code': '30037591', 'managers_id': 1280, 'funds_history_update_time': datetime.datetime(2017, 8, 4, 15, 4, 57), 'managers_gendar_guess': None, 'working_since_date': datetime.date(2007, 5, 11), 'managers_photo': None}
30037591


,基金代码,基金名称,相关链接,基金类型,规模（亿元）,任职时间,任职天数,任职回报,start_date,end_date,...,funds_type_x,funds_start_date_x,update_time_x,historical_price_time_x,funds_id_y,funds_name_full_y,funds_type_y,funds_start_date_y,update_time_y,historical_price_time_y
0,1696,南方智造股票,估值图基金吧档案,股票型,4.44,2017-08-03 ~ 至今,1天,0.00%,2017-08-03,9999-9-9,...,股票型,2017-08-03,2017-08-04 14:08:11,2017-08-04 14:09:04,120052,南方智造股票,股票型,2017-08-03,2017-08-04 14:08:11,2017-08-04 14:09:04
1,202213,南方安心,估值图基金吧档案,保本型,37.09,2016-10-10 ~ 至今,298天,-1.07%,2016-10-10,9999-9-9,...,保本型,2012-12-21,2017-08-03 16:51:08,2017-08-04 06:22:14,1508,南方安心,保本型,2012-12-21,2017-08-03 16:51:08,2017-08-04 06:22:14
2,2160,南方驱动混合,估值图基金吧档案,混合型,1.99,2016-03-23 ~ 至今,1年又134天,25.40%,2016-03-23,9999-9-9,...,混合型,2016-03-23,2017-08-04 12:35:19,2017-08-04 09:11:05,3505,南方驱动混合,混合型,2016-03-23,2017-08-04 12:35:19,2017-08-04 09:11:05
3,202001,南方稳健成长,估值图基金吧档案,混合型,21.33,2012-11-23 ~ 至今,4年又255天,91.24%,2012-11-23,9999-9-9,...,混合型,2001-09-28,2017-08-04 10:52:18,2017-08-04 08:22:03,3014,南方稳健成长,混合型,2001-09-28,2017-08-04 10:52:18,2017-08-04 08:22:03
4,202002,南方稳健成长贰号,估值图基金吧档案,混合型,19.36,2012-11-23 ~ 至今,4年又255天,92.42%,2012-11-23,9999-9-9,...,混合型,2006-07-25,2017-08-04 10:47:09,2017-08-04 08:16:09,2959,南方稳健成长贰号,混合型,2006-07-25,2017-08-04 10:47:09,2017-08-04 08:16:09
5,202101,南方宝元债券,估值图基金吧档案,债券型,15.89,2010-12-02 ~ 2016-03-30,5年又120天,63.72%,2010-12-02,2016-03-30,...,债券型,2002-09-20,2017-08-03 22:13:08,2017-08-04 07:54:13,2742,南方宝元债券,债券型,2002-09-20,2017-08-03 22:13:08,2017-08-04 07:54:13
6,202101,南方宝元债券,估值图基金吧档案,债券型,15.89,2007-05-11 ~ 2009-02-12,1年又278天,4.51%,2007-05-11,2009-02-12,...,债券型,2002-09-20,2017-08-03 22:13:08,2017-08-04 07:54:13,2742,南方宝元债券,债券型,2002-09-20,2017-08-03 22:13:08,2017-08-04 07:54:13
7,202005,南方成份精选混合,估值图基金吧档案,混合型,42.17,2007-05-14 ~ 2012-11-23,5年又195天,-20.46%,2007-05-14,2012-11-23,...,混合型,2007-05-14,2017-08-03 07:52:08,2017-08-04 11:10:03,4660,南方成份精选混合,混合型,2007-05-14,2017-08-03 07:52:08,2017-08-04 11:10:03


INSERT INTO `managers_2_funds_history` (managers_id, funds_id, history_start_date, history_end_date) VALUES (1280, 120052, 2017-08-03, 9999-9-9) ON DUPLICATE KEY UPDATE history_end_date=9999-9-9;
INSERT INTO `managers_2_funds_history` (managers_id, funds_id, history_start_date, history_end_date) VALUES (1280, 1508, 2016-10-10, 9999-9-9) ON DUPLICATE KEY UPDATE history_end_date=9999-9-9;
INSERT INTO `managers_2_funds_history` (managers_id, funds_id, history_start_date, history_end_date) VALUES (1280, 3505, 2016-03-23, 9999-9-9) ON DUPLICATE KEY UPDATE history_end_date=9999-9-9;
INSERT INTO `managers_2_funds_history` (managers_id, funds_id, history_start_date, history_end_date) VALUES (1280, 3014, 2012-11-23, 9999-9-9) ON DUPLICATE KEY UPDATE history_end_date=9999-9-9;
INSERT INTO `managers_2_funds_history` (managers_id, funds_id, history_start_date, history_end_date) VALUES (1280, 2959, 2012-11-23, 9999-9-9) ON DUPLICATE KEY UPDATE history_end_date=9999-9-9;
INSERT INTO `managers_2_fund